mne.io.read_raw_edf: Loads the file and read the data

raw.info 

In [ ]:
import sys
print(sys.executable)

%matplotlib qt


In [ ]:
import mne
import sys
from mne.preprocessing import ICA

print(f"Using Python executable: {sys.executable}")

# MNE will automatically download this file and store it in your home directory
# in a folder called 'mne_data'.
physionet_file_path = r'C:\Users\524yu\OneDrive\Documents\VSCODEE\BMI-Robotic-Control\physionet.org\files\eegmmidb\1.0.0\S001\S001R01.edf'

# Load the EEG data
# preload=True loads the data into memory
raw = mne.io.read_raw_edf(physionet_file_path, preload=True)

# Print basic information about the file
print("\n--- Data Info ---")
print(raw.info)


channel_subgroups = {
    "Frontal": [ch for ch in raw.ch_names if ch.startswith(("F", "Fp"))],
    "Central": [ch for ch in raw.ch_names if ch.startswith(("C"))],
    "Parietal": [ch for ch in raw.ch_names if ch.startswith(("P"))],
    "Occipital": [ch for ch in raw.ch_names if ch.startswith(("O"))],
    "Temporal": [ch for ch in raw.ch_names if ch.startswith(("T", "Tp"))]
}


# List available channel names
print ("--Channel Names--")

for regions, channels in channel_subgroups.items():
    if not channels:
        continue
    
    print (f"\n {regions}, Region: ({len(channels)} channels): ")
    print (f", ".join(channels))


# Show duration
duration_in_seconds = raw.n_times / raw.info['sfreq']
print(f"\nDuration: {duration_in_seconds:.2f} seconds")










Grab Plotting of all regions before filtering
1. Visualise raw EEG signal in time domain.
2. Visualise the power spectral density of the EEG signal (how signal power is distributed across different frequncy)

In [ ]:
print("Showing all channels together...")
raw.plot(
    duration=5,            # how many seconds per screen
    n_channels=len(raw.ch_names),  # display all channels
    scalings='auto',       # automatically scale signal amplitude
    title='All EEG Channels',
    butterfly=True         # overlay channels for better comparison
)

psd = raw.compute_psd(fmax=60)
psd.plot(picks="eeg", exclude = "bads", average = True, dB = True)
# Pick - picks what type of signal it has to plot; eeg signal
# exlude - do not include bad channels
# average -


In [ ]:
raw.filter(l_freq=1., h_freq=40.)

Plotting all of the channels together after using the filter
- the regular raw.plot is unable to view the difference clearly

psd compute is used to see a better comparison when the filter is applied 
- View signal power distribution across different frequencies (filtering affects this graph)

In [ ]:
print("Showing all channels together...")
raw.plot(
    duration=5,            # how many seconds per screen
    n_channels=len(raw.ch_names),  # display all channels
    scalings='auto',       # automatically scale signal amplitude
    title='All EEG Channels',
    butterfly=True         # overlay channels for better comparison
)

psd = raw.compute_psd(fmax=60)
psd.plot(picks = "eeg",exclude="bads", average= True, dB = True)

Graph plotting of separate regions

In [ ]:
# Plot a short segment to visualize
# This will open an interactive plot window.

for regions, channels in channel_subgroups.items():
    if not channels:
        continue
    print (f"showing regions: {regions}\n Channels shown: {len(channels)} channels: {channels}")
    raw.plot(picks = channels, duration= 5, n_channels=len(channels), scalings = "auto", title = f"{regions} Region EEG")


Preprocessing takes place
ICA process starts (Independant Component Analysis)

In [ ]:
ica = ICA(n_components=20, random_state=97, max_iter=800)
ica.fit(raw)

ica.plot_sources(raw)
ica.exclude = [0, 4, 11]

# IMPORTANT: Always apply the cleaning to a copy of the data
raw_cleaned = raw.copy()
ica.apply(raw_cleaned)

print("Displaying original data...")
raw.plot(title="Original Data")

print("\nDisplaying data after removing artifacts...")
raw_cleaned.plot(title="Cleaned Data")
